### Include Librairies

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
from datetime import *
import pygal             # Usefull for french map
from pygal.style import Style
import matplotlib.pyplot as plt
import requests
import json
import csv

%matplotlib inline

### Import Data

In [2]:
df = pd.read_csv('data.csv')

In [ ]:
df.head(5)

### Check row 

In [4]:
df['activitePrincipaleEtablissement'].count()

998

### Slipt Columns

In [5]:
columns_int = df.select_dtypes(include=['int']).columns
columns_int

Index(['siren', 'nic', 'siret', 'nombrePeriodesEtablissement',
       'categorieJuridiqueUniteLegale', 'nicSiegeUniteLegale',
       'codePostalEtablissement', 'codeCommuneEtablissement'],
      dtype='object')

In [6]:
columns_float = df.select_dtypes(include=['float']).columns
columns_float

Index(['anneeEffectifsEtablissement', 'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale', 'pseudonymeUniteLegale',
       'identifiantAssociationUniteLegale', 'anneeEffectifsUniteLegale',
       'anneeCategorieEntreprise', 'numeroVoieEtablissement',
       'libelleCommuneEtrangerEtablissement', 'codeCedexEtablissement',
       'codePaysEtrangerEtablissement', 'libellePaysEtrangerEtablissement',
       'complementAdresse2Etablissement', 'numeroVoie2Etablissement',
       'indiceRepetition2Etablissement', 'codePostal2Etablissement',
       'libelleCommuneEtranger2Etablissement',
       'distributionSpeciale2Etablissement', 'codeCommune2Etablissement',
       'codeCedex2Etablissement', 'libelleCedex2Etablissement',
       'codePaysEtranger2Etablissement', 'libellePaysEtranger2Etablissement',
       'dateFin', 'enseigne2Etablissement', 'enseigne3Etablissement'],
      dtype='object')

In [7]:
columns_object = df.select_dtypes(include=['object']).columns
columns_object

Index(['statutDiffusionEtablissement', 'dateCreationEtablissement',
       'trancheEffectifsEtablissement',
       'activitePrincipaleRegistreMetiersEtablissement',
       'dateDernierTraitementEtablissement', 'etatAdministratifUniteLegale',
       'statutDiffusionUniteLegale', 'unitePurgeeUniteLegale',
       'dateCreationUniteLegale', 'denominationUniteLegale',
       'sigleUniteLegale', 'denominationUsuelle1UniteLegale',
       'sexeUniteLegale', 'nomUniteLegale', 'nomUsageUniteLegale',
       'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale',
       'prenom4UniteLegale', 'prenomUsuelUniteLegale',
       'activitePrincipaleUniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale',
       'economieSocialeSolidaireUniteLegale', 'caractereEmployeurUniteLegale',
       'trancheEffectifsUniteLegale', 'dateDernierTraitementUniteLegale',
       'categorieEntreprise', 'complementAdresseEtablissement',
       'indiceRepetitionEtablissement', 'typeVoieEtablissement',
   

### Nombre d'entreprise differentes :

In [8]:
df['siren'].nunique()

879

### Nombre de Ligne :

In [9]:
df['siren'].count()

1000

### Verification du nombre de Siret
qui doit etre égale au nombre de ligne

In [10]:
df['siret'].nunique()

1000

### Ont groupebt('siren') pour verifier les information precedentes
c'est regrouper par entreprise

In [11]:
df.groupby('siren').first().count()

nic                                               879
siret                                             879
statutDiffusionEtablissement                      879
dateCreationEtablissement                         160
trancheEffectifsEtablissement                     423
anneeEffectifsEtablissement                        27
activitePrincipaleRegistreMetiersEtablissement      4
dateDernierTraitementEtablissement                122
etablissementSiege                                879
nombrePeriodesEtablissement                       879
etatAdministratifUniteLegale                      879
statutDiffusionUniteLegale                        879
unitePurgeeUniteLegale                            740
dateCreationUniteLegale                           879
categorieJuridiqueUniteLegale                     879
denominationUniteLegale                           177
sigleUniteLegale                                    5
denominationUsuelle1UniteLegale                    17
denominationUsuelle2UniteLeg

## Methode 1 : NAF, APE

### Function Mean Date / Print Info

In [ ]:
def mean_date(data):
    tot_date = 0
    count = 0
    
    for value in data:
        try:
            tot_date += value.year
            count += 1
        except:
            pass
    if count != 0:
        return int(tot_date/count)
    else:
        return 0

In [13]:
def print_infos(name_value,nb_value,mean_date_value,name_code,max_code):
    
    total = 0
    
    for i in range(len(name_value)):
        total += nb_value[i]
        try:
            l = i * 3 
            print("-Name : %s, \tTotal : %s, mean_date : %s, \tCode Commune (3): %s,%s,%s, nb_entreprises : %s,%s,%s, Département : %s,%s,%s\n" %(name_value[i],nb_value[i],mean_date_value[i],name_code[l],name_code[l+1],name_code[l+2],max_code[l],max_code[l+1],max_code[l+2],code_departement[l],code_departement[l+1],code_departement[l+2]))
               
        except:
            pass
        
    print("\nTotal siren : ",total)
        

### Find main Region foreach activitie

In [14]:
def max_region(data,df):
    
    list_code_commune = list()
    save_code_commune = list()
    save_count = list()
    final_save_count = list()
    final_name_code = list()

    for i in range(len(df)):
        try:
            list_code_commune.append(data[i])
        except:
            pass
          
    for i in range(len(list_code_commune)):
        count = 0
        for j in range(i,len(list_code_commune)):
            if list_code_commune[i] == list_code_commune[j]:
                count += 1
        if list_code_commune[i] not in save_code_commune:
            save_code_commune.append(list_code_commune[i])
            save_count.append(count)
       
    try:
        final_save_count.append(max(save_count))
        max_save_count = final_save_count[-1]
        nb_max = 3
    
        save = save_count[0]
    
        for j in range(nb_max-1):
            for i in range(len(save_count)):
                if save_count[i] > save and save_count[i] < max_save_count:
                    save = save_count[i]
            final_save_count.append(save)
            max_save_count = save
            save = save_count[0]
    except:
        pass
     
    for value in final_save_count:
        try:
            m_index = [i for i, j in enumerate(save_count) if j == value ]
            final_name_code.append(save_code_commune[m_index[0]])
            
        except:
            pass
            
    return final_name_code,final_save_count


### Function chiffres : Split Code_Commune

In [15]:
def chiffres(n):
    c = []
    while n >= 1:
        c.append(int(n % 10))
        n = n / 10
        
    save = 0
    for i in range(int(len(c)/2)):
        save = c[i]
        c[i] = c[-i -1]
        c[-i-1] = save
    return c

### Convert Code_Commune to Code_Departement

In [16]:
def code_depart(code):
    code_departement = []
    try:
        c = chiffres(int(code))
        #print(c)
        code_departement.append(c[0]*10 +c[1])

    except:
        pass
    
    return code_departement


### Print Info Nomenclature Activite Principale Etablissement

In [17]:
tot = 0
name_value = list()
nb_value = list()
mean_date_value = list()
name_code = list()
max_code = list()
code_departement = list()

col_nomenclature = df['activitePrincipaleEtablissement'].unique()

for value in col_nomenclature:
    tot_imp = len(df[df['activitePrincipaleEtablissement'] == value].groupby('siren').first())

    #meandate = mean_date(df[df['activitePrincipaleEtablissement'] == value]['dateCreationEtablissement'])
    code,m = max_region(df[df['activitePrincipaleEtablissement'] == value]['codeCommuneEtablissement'],df)
    
    meandate= 0
    # Save Infos
    name_value.append(value)
    nb_value.append(tot_imp)
    mean_date_value.append(meandate)
    
    for i in range(len(code)):
        name_code.append(code[i])
        max_code.append(m[i])
    
    
    for i in range(len(code)):
        try:
            c = chiffres(int(code[i]))
            #print(c)
            code_departement.append(c[0]*10 +c[1])

        except:
            pass
    

# Print infos
print_infos(name_value,nb_value,mean_date_value,name_code,max_code)



-Name : 32.12Z, 	Total : 14, mean_date : 0, 	Code Commune (3): 97611,97608,98727.0, nb_entreprises : 7,2,1, Département : 97,97,98

-Name : 47.89Z, 	Total : 150, mean_date : 0, 	Code Commune (3): 97611,97615,80001.0, nb_entreprises : 93,12,6, Département : 97,97,80

-Name : 95.24Z, 	Total : 2, mean_date : 0, 	Code Commune (3): 54528.0,54528.0,54528.0, nb_entreprises : 1,1,1, Département : 54,54,54

-Name : 85.59A, 	Total : 6, mean_date : 0, 	Code Commune (3): 97611,2691.0,2691.0, nb_entreprises : 2,1,1, Département : 97,26,26

-Name : 22.02, 	Total : 57, mean_date : 0, 	Code Commune (3): 44057.0,80765.0,80765.0, nb_entreprises : 2,1,1, Département : 44,80,80

-Name : 79.06, 	Total : 150, mean_date : 0, 	Code Commune (3): 06004,21231,47001, nb_entreprises : 24,23,18, Département : 60,21,47

-Name : 64.42, 	Total : 55, mean_date : 0, 	Code Commune (3): 47001,80025.0,80025.0, nb_entreprises : 4,2,2, Département : 47,80,80

-Name : 70.2C, 	Total : 1291, mean_date : 0, 	Code Commune (3): 21

## Methode 2 : NAF, APE

## Class API Data.Gouv info Siret

In [12]:
class info_by_siret(object):
    
    def __init__(self,n_siret):
        self.n_siret = n_siret
    
    def connect(self):
        response = requests.get('https://entreprise.data.gouv.fr/api/sirene/v1/siret/'+self.n_siret)
        print("response code :",response.status_code)
        x = response.text
        x = json.loads(x)
        df = pd.read_json(json.dumps(x))
        df = np.transpose(df)
        self.df = df
        

In [13]:
c = info_by_siret('21310555400017')
c.connect()
c.df['tranche_effectif_salarie_entreprise_centaine_pret']

response code : 200


etablissement    12400
Name: tranche_effectif_salarie_entreprise_centaine_pret, dtype: object

### Convert dateCreationEtablissement to Datatime

In [14]:
df['dateCreationEtablissement'] = pd.to_datetime(df['dateCreationEtablissement'])

### Class NAF

In [15]:
from collections import Counter

class naf(object):
    
    def __init__(self):
        self.nb_row = df['activitePrincipaleEtablissement'].nunique()
        self.name_activite = df['activitePrincipaleEtablissement'].unique()
        self.result = []
        self.final_result = []
        
    def __print__(self,name,total,departement,nb_emp):
        print("-Name: %s, Nb_entreprises_total: %s, Departement: %s,-> Nb_inside: %s,employee: %s" %(name,total,departement[0],departement[1],nb_emp))
        print("--------------")
    
    def mean_date(self,data):
        total = 0
        index = 0
        for i in range(len(data)):
            try:
                total += data[i].year
                index += 1
            except:
                pass
        mean = int(total/index)
        print(mean)
        
        
    def split(self,n):
        c = []
        while n >= 1:
            c.append(int(n % 10))
            n = n / 10

        save = 0
        for i in range(int(len(c)/2)):
            save = c[i]
            c[i] = c[-i -1]
            c[-i-1] = save

        return c[0]*10 +c[1]
         
    def first_departement(self,data):
        data_split = []
        code_commune = []
        for val in data:
            data_split.append(self.split(val))
            code_commune.append(val)
        m = Counter(data_split).most_common(1)
        return m
    
    def func_result(self):
        passage = []
        for i in range(len(self.result)):
            if self.result[i][2] not in passage:
                save = self.result[i]
                for j in range(i,len(self.result)):
                    if save[2] == self.result[j][2]:
                        save = save[0]+", "+ self.result[j][0] , save[1]+self.result[j][1],save[2],save[3]+self.result[j][3],save[4]+ self.result[j][4]

                
                self.final_result.append(save)
                passage.append(self.result[i][2])
     
    
    def nb_employee(self,data,departement):
        self.total_employee = 0
        siret = 0
        #print(len(data))
        for val,val2 in zip(data['codeCommuneEtablissement'],data['siret']):
            try:
                
                code_departement = self.split(val)
                if code_departement == departement[0]:
                    response = requests.get('https://entreprise.data.gouv.fr/api/sirene/v1/siret/21310555400017')#+val2)
                    x = response.text
                    x = json.loads(x)
                    df = pd.read_json(json.dumps(x))
                    df = np.transpose(df)
                    self.df = df
                    self.total_employee += int(self.df['tranche_effectif_salarie_entreprise_centaine_pret'][0])
            except:
                pass

    def main(self):
        tot = 0
        for value in self.name_activite:
                
                total = len(df[df['activitePrincipaleEtablissement'] == value].groupby('siren').first())
                
            
                #if df[df['activitePrincipaleEtablissement'] == value]['codeCommuneEtablissement']
                try:
                    departement = self.first_departement(df[df['activitePrincipaleEtablissement'] == value]['codeCommuneEtablissement'])
                    departement = departement[0]
                    
                    self.nb_employee(df[df['activitePrincipaleEtablissement'] == value][['codeCommuneEtablissement','siret']],departement)
                    final = (value,total,departement[0],departement[1],self.total_employee)
                    
                    self.result.append(final)
                    print(final)
                    self.__print__(value,total,departement,self.total_employee)
                    
                    
                except:
                    pass
        self.func_result()    

### Use class NAF
format : (Activité', Nb_entreprise, Département, Nb_inside, Nb_employees)

In [16]:
c = naf()
c.main()

('32.12Z', 1, 13, 3, 37200)
-Name: 32.12Z, Nb_entreprises_total: 1, Departement: 13,-> Nb_inside: 3,employee: 37200
--------------
('47.89Z', 1, 84, 1, 12400)
-Name: 47.89Z, Nb_entreprises_total: 1, Departement: 84,-> Nb_inside: 1,employee: 12400
--------------
('95.24Z', 1, 54, 1, 12400)
-Name: 95.24Z, Nb_entreprises_total: 1, Departement: 54,-> Nb_inside: 1,employee: 12400
--------------
('85.59A', 1, 26, 1, 12400)
-Name: 85.59A, Nb_entreprises_total: 1, Departement: 26,-> Nb_inside: 1,employee: 12400
--------------
('22.02', 7, 80, 6, 74400)
-Name: 22.02, Nb_entreprises_total: 7, Departement: 80,-> Nb_inside: 6,employee: 74400
--------------
('79.06', 4, 44, 2, 24800)
-Name: 79.06, Nb_entreprises_total: 4, Departement: 44,-> Nb_inside: 2,employee: 24800
--------------
('64.42', 10, 80, 7, 86800)
-Name: 64.42, Nb_entreprises_total: 10, Departement: 80,-> Nb_inside: 7,employee: 86800
--------------
('70.2C', 47, 44, 26, 322400)
-Name: 70.2C, Nb_entreprises_total: 47, Departement: 44,-

('93.0D', 1, 42, 1, 12400)
-Name: 93.0D, Nb_entreprises_total: 1, Departement: 42,-> Nb_inside: 1,employee: 12400
--------------
('64.49', 12, 80, 7, 86800)
-Name: 64.49, Nb_entreprises_total: 12, Departement: 80,-> Nb_inside: 7,employee: 86800
--------------
('59.08', 3, 41, 2, 24800)
-Name: 59.08, Nb_entreprises_total: 3, Departement: 41,-> Nb_inside: 2,employee: 24800
--------------
('62.44', 10, 80, 6, 74400)
-Name: 62.44, Nb_entreprises_total: 10, Departement: 80,-> Nb_inside: 6,employee: 74400
--------------
('60.2B', 2, 41, 1, 12400)
-Name: 60.2B, Nb_entreprises_total: 2, Departement: 41,-> Nb_inside: 1,employee: 12400
--------------
('45.20A', 2, 80, 2, 24800)
-Name: 45.20A, Nb_entreprises_total: 2, Departement: 80,-> Nb_inside: 2,employee: 24800
--------------
('47.78C', 1, 40, 1, 12400)
-Name: 47.78C, Nb_entreprises_total: 1, Departement: 40,-> Nb_inside: 1,employee: 12400
--------------
('18.11', 1, 13, 1, 12400)
-Name: 18.11, Nb_entreprises_total: 1, Departement: 13,-> Nb_i

('18.2D', 1, 80, 1, 12400)
-Name: 18.2D, Nb_entreprises_total: 1, Departement: 80,-> Nb_inside: 1,employee: 12400
--------------
('57.09', 2, 80, 1, 12400)
-Name: 57.09, Nb_entreprises_total: 2, Departement: 80,-> Nb_inside: 1,employee: 12400
--------------
('47.02', 1, 42, 1, 12400)
-Name: 47.02, Nb_entreprises_total: 1, Departement: 42,-> Nb_inside: 1,employee: 12400
--------------
('64.44', 2, 40, 1, 12400)
-Name: 64.44, Nb_entreprises_total: 2, Departement: 40,-> Nb_inside: 1,employee: 12400
--------------
('47.73Z', 2, 40, 1, 12400)
-Name: 47.73Z, Nb_entreprises_total: 2, Departement: 40,-> Nb_inside: 1,employee: 12400
--------------
('00.98', 3, 41, 1, 12400)
-Name: 00.98, Nb_entreprises_total: 3, Departement: 41,-> Nb_inside: 1,employee: 12400
--------------
('39.01', 1, 41, 1, 12400)
-Name: 39.01, Nb_entreprises_total: 1, Departement: 41,-> Nb_inside: 1,employee: 12400
--------------
('55.1A', 4, 41, 3, 37200)
-Name: 55.1A, Nb_entreprises_total: 4, Departement: 41,-> Nb_inside:

('28.5C', 1, 80, 1, 12400)
-Name: 28.5C, Nb_entreprises_total: 1, Departement: 80,-> Nb_inside: 1,employee: 12400
--------------
('51.8P', 1, 80, 2, 24800)
-Name: 51.8P, Nb_entreprises_total: 1, Departement: 80,-> Nb_inside: 2,employee: 24800
--------------
('46.61Z', 1, 80, 2, 24800)
-Name: 46.61Z, Nb_entreprises_total: 1, Departement: 80,-> Nb_inside: 2,employee: 24800
--------------
('51.6N', 1, 80, 1, 12400)
-Name: 51.6N, Nb_entreprises_total: 1, Departement: 80,-> Nb_inside: 1,employee: 12400
--------------
('58.09', 2, 40, 1, 12400)
-Name: 58.09, Nb_entreprises_total: 2, Departement: 40,-> Nb_inside: 1,employee: 12400
--------------
('93.19Z', 1, 40, 1, 12400)
-Name: 93.19Z, Nb_entreprises_total: 1, Departement: 40,-> Nb_inside: 1,employee: 12400
--------------
('52.4E', 2, 42, 1, 12400)
-Name: 52.4E, Nb_entreprises_total: 2, Departement: 42,-> Nb_inside: 1,employee: 12400
--------------
('15.01', 1, 41, 1, 12400)
-Name: 15.01, Nb_entreprises_total: 1, Departement: 41,-> Nb_insid

('14.1A', 1, 40, 1, 12400)
-Name: 14.1A, Nb_entreprises_total: 1, Departement: 40,-> Nb_inside: 1,employee: 12400
--------------
('57.04', 1, 40, 1, 12400)
-Name: 57.04, Nb_entreprises_total: 1, Departement: 40,-> Nb_inside: 1,employee: 12400
--------------
('01.11Z', 1, 42, 1, 12400)
-Name: 01.11Z, Nb_entreprises_total: 1, Departement: 42,-> Nb_inside: 1,employee: 12400
--------------


### Final Result, secteurs d'activitées regroupé par département
format : ('Activité', Nb_entreprise, Département, Nb_inside, Nb_employees)

In [17]:
c.final_result

[('32.12Z, 32.12Z, 18.11, 46.19B', 4, 13, 8, 99200),
 ('47.89Z, 47.89Z', 2, 84, 2, 24800),
 ('95.24Z, 95.24Z', 2, 54, 2, 24800),
 ('85.59A, 85.59A', 2, 26, 2, 24800),
 ('22.02, 22.02, 64.42, 57.11, 86.06, 55.4B, 67.05, 64.45, 65.06, 57.03, 38.40, 74.1J, 67.07, 65.03, 50.1Z, 65.02, 62.21, 46.69B, 59.02, 45.3F, 52.4Y, 28.6F, 52.4N, 10.81Z, 58.12, 51.2A, 20.01, 44.33, 64.49, 62.44, 45.20A, 57.02, 82.06, 64.24, 64.47, 55.2C, 56.02, 78.02, 60.2E, 60.2L, 21.11, 92.7C, 15.11, 25.72Z, 63.1E, 17.21A, 59.10, 20.30Z, 64.13, 46.74B, 37.1Z, 55.60, 18.2D, 57.09, 52.6E, 47.08, 01.42Z, 64.43, 64.25, 46.73A, 28.14Z, 45.2V, 46.69C, 10.61A, 49.39A, 21.12, 28.5C, 51.8P, 46.61Z, 51.6N, 57.01, 66.04, 45.23, 55.4A, 01.4A, 52.4R, 15.7A, 57.05, 08.11Z, 86.10Z, 20.5A, 16.23Z, 20.3Z, 67.06, 67.12, 68.32A, 28.7L, 25.99A, 29.5N, 25.73A, 49.41A, 41.10A, 47.52A, 46.74A, 10.91Z, 16.21Z, 25.61Z, 88.02',
  256,
  80,
  199,
  2467600),
 ('79.06, 79.06, 70.2C, 62.11, 62.43, 64.11, 55.40, 62.41, 67.04, 64.12, 81.21, 64.4

# French map : Use Librairie PYGAL

### Custom Style from map

In [18]:
custom_style = Style(
  background='transparent',
  transition='200ms ease-in',
)

### Function which draws chart

In [19]:
def chart():
    fr_chart = pygal.maps.fr.Departments(style=custom_style,height=450)    #,legend_at_bottom=True)
    fr_chart.title = 'Activitie by departments'
    for i in range(len(c.final_result)):
        try:    
            fr_chart.add(c.final_result[i][0], {c.final_result[i][2] : c.final_result[i][3]})    
        except:
            pass
    fr_chart.render_in_browser() # Exporte to Browser

In [20]:
chart()

file:///var/folders/v3/6rms4ljn2dzdln_l8746trf00000gn/T/tmpgvk2hnx7.html
